In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

from scipy import stats

# from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

In [ ]:
data_audi = pd.read_csv('D:/Data/Car Price/audi.csv')
data_bmw = pd.read_csv('D:/Data/Car Price/bmw.csv')
data_ford = pd.read_csv('D:/Data/Car Price/ford.csv')
data_hyundi = pd.read_csv('D:/Data/Car Price/hyundi.csv').rename(columns= {'tax(£)': 'tax'})
data_merc = pd.read_csv('D:/Data/Car Price/merc.csv')
data_skoda = pd.read_csv('D:/Data/Car Price/skoda.csv')
data_toyota = pd.read_csv('D:/Data/Car Price/toyota.csv')

In [ ]:
all_data = {'audi': data_audi, 'bmw': data_bmw, 'ford': data_ford, 'hyundi': data_hyundi, 'merc': data_merc, 'skoda': data_skoda, 'toyota': data_toyota}

In [ ]:
# Add new feature "brand"
for key, dataset in all_data.items():
    dataset['brand'] = key

# Merge all dataframes into one
data_temp = pd.DataFrame(columns=data_audi.columns)

for key, df in all_data.items():
    data_temp = data_temp.append(df, ignore_index=True)

# Shuffle final dataframe
data = data_temp.sample(frac=1).reset_index(drop=True)

In [ ]:
data

In [ ]:
data.describe()

In [ ]:
data.info()

# Data cleaning

## Outliers

In [ ]:
fig = go.Figure()

fig.add_trace(go.Box(
    y = data['price'],
    marker_color='rgb(9,56,125)',
    name = 'Price'
))

fig.update_layout(
    title_text = 'Price box plot',
    template = 'plotly_dark'
)

There are many points that looks like outliers but for me it is completely normal that some cars are very expensive. There is no need to delete these records.

In [ ]:
fig = go.Figure()

fig.add_trace(go.Box(
    y = data['mileage'],
    marker_color='rgb(128, 0, 0)',
    name = 'Mileage'
))

fig.update_layout(
    title_text = 'Mileage box plot',
    template = 'plotly_dark'
)

In [ ]:
fig = go.Figure()

fig.add_trace(go.Box(
    y = data['tax'],
    marker_color='rgb(0, 153, 51)',
    name = 'Tax'
))

fig.update_layout(
    title_text = 'Tax box plot',
    template = 'plotly_dark'
)

In [ ]:
fig = go.Figure()

fig.add_trace(go.Box(
    y = data['mpg'],
    marker_color='rgb(153, 92, 0)',
    name = 'Miles per Gallon'
))

fig.update_layout(
    title_text = 'Miles per Gallon box plot',
    template = 'plotly_dark'
)

Tax and mileage looks strange but it is still something I can believe in but outliers from mpg are completely unrealistic. I am going to delete car with 470 mpg from our dataset. 

In [ ]:
data = data[data['mpg'] < 470]

In [ ]:
fig = go.Figure()

fig.add_trace(go.Box(
    y = data['mpg'],
    marker_color='rgb(153, 92, 0)',
    name = 'Miles per Gallon'
))

fig.update_layout(
    title_text = 'Miles per Gallon box plot',
    template = 'plotly_dark'
)

## Columns usefulnes
We are going to replace very rare models with 'Rare' value to reduce number of new dimensions during dummy variables creation and to make dataset a little cleaner.

In [ ]:
to_replace = data['model'].value_counts()[data['model'].value_counts().values < 10]
to_replace

In [ ]:
indexes = data.index[data['model'].isin(to_replace.index)]
data.loc[indexes, 'model'] = ' Rare'

# Single variable analysis

In [ ]:
fig = go.Figure()

to_plot = data['year'].value_counts()

fig.add_trace(go.Bar(
    x = to_plot.index,
    y = to_plot.values
))

fig.update_layout(
    title_text = 'Number of cars from every year',
    template = 'plotly_dark',
    xaxis_title = 'Year',
    yaxis_title = 'Number of cars'
)

In [ ]:
fig = go.Figure()

fig.add_trace(go.Histogram(
    x = data['mileage'],
    nbinsx = 170
))

fig.update_layout(
    title_text = 'Distribution of mileage feature',
    template = 'plotly_dark',
    xaxis_title = 'Mileage'
)

In [ ]:
fig = go.Figure()

fig.add_trace(go.Histogram(
    x = data['price'],
    nbinsx = 170
))

fig.update_layout(
    title_text = 'Distribution of price feature',
    template = 'plotly_dark',
    xaxis_title = 'Price'
)

In [ ]:
fig = go.Figure()

night_colors = ['rgb(56, 75, 126)', 'rgb(18, 36, 37)', 'rgb(34, 53, 101)',
                'rgb(36, 55, 57)', 'rgb(6, 4, 4)']

to_plot = data['transmission'].value_counts()

fig.add_trace(go.Pie(
    values = to_plot.values,
    labels = to_plot.index,
    textinfo = 'label+percent',
    marker_colors = night_colors
))

fig.update_layout(
    title_text = 'Transmission Pie Plot',
    template = 'plotly_dark'
)

In [ ]:
fig = go.Figure()

to_plot = data['brand'].value_counts()

fig.add_trace(go.Pie(
    values = to_plot.values,
    labels = to_plot.index,
    textinfo = 'label+percent'
))

fig.update_layout(
    title_text = 'Brand Pie Plot',
    template = 'plotly_dark'
)

In [ ]:
fig = go.Figure()

to_plot = data['fuelType'].value_counts()

fig.add_trace(go.Bar(
    y = to_plot.index,
    x = to_plot.values,
    orientation = 'h',
    text = to_plot.values,
    textposition = 'outside',
))

fig.update_layout(
    autosize = False,
    width = 850,
    template = 'plotly_dark',
    title_text = 'Fuel Type Bar Plot'
)

# Multiple variables analysis

In [ ]:
data

### Pearson
<b>The Pearson correlation coefficient<b> is a measure of the linear relationship between two features. It's the ratio of the covariance of x and y to the product of their standard deviations. It's often denoted with the letter r and called Pearson's r.

### Spearman
The Spearman correlation coefficient between two features is the Pearson correlation coefficient between their rank values. It's calculated the same ways as the Pearson correlation coefficient but takes into account their ranks instead of their values.

### Kendall
The Kendall correlation coefficient compares the number of concordant and discordant pairs of data. This coefficient is basen on the differce in the counts of concordant and discordant pairs relative to the number of x-y paris.

concordant if either (xᵢ > xⱼ and yᵢ > yⱼ) or (xᵢ < xⱼ and yᵢ < yⱼ) <br>
discordant if either (xᵢ < xⱼ and yᵢ > yⱼ) or (xᵢ > xⱼ and yᵢ < yⱼ) <br>
neither if there’s a tie in x (xᵢ = xⱼ) or a tie in y (yᵢ = yⱼ)

In [ ]:
def correlation(x, y):
    """
    x, y - two lists of features
    This function prints Pearson, Spearman
    """

    # Pearson's r
    pearson_r = np.corrcoef(x, y)[0, 1]

    # Spearman's rho
    spearman_rho = stats.spearmanr(x, y).correlation

    # Kendall's tau
    kendall_tau = stats.kendalltau(x, y).correlation

    print('Pearson correlation coefficient: {}'.format(pearson_r))
    print('Spearman correlation coefficient: {}'.format(spearman_rho))
    print('Kendall correlation coefficient: {}'.format(kendall_tau))

In [ ]:
fig = go.Figure()

to_plot = data.sample(frac=0.10, random_state=42)

fig.add_trace(go.Scatter(
    x = to_plot['price'],
    y = to_plot['mileage'],
    mode = 'markers'
))

fig.update_layout(
    title_text = 'Mileage x Price Scatter Plot',
    template = 'plotly_dark',
    xaxis_title = 'Price',
    yaxis_title = 'Mileage'
)

In [ ]:
correlation(data['price'].tolist(), data['mileage'].tolist())

In [ ]:
fig = go.Figure()

to_plot = data.sample(frac=0.10, random_state=42)

fig.add_trace(go.Scatter(
    x = to_plot['price'],
    y = to_plot['mpg'],
    mode = 'markers'
))

fig.update_layout(
    title_text = 'Miles per gallon',
    template = 'plotly_dark',
    xaxis_title = 'Price',
    yaxis_title = 'Miles per gallon'
)

In [ ]:
correlation(data['price'].tolist(), data['mpg'].tolist())

In [ ]:
data_price_float = data.copy()
data_price_float['price'] = data_price_float['price'].map(lambda x: float(x))

In [ ]:
fig = go.Figure()

to_plot = data_price_float.groupby(by='transmission', as_index=False).mean()

fig.add_trace(go.Bar(
    y = to_plot['transmission'],
    x = to_plot['price'],
    orientation='h'
))

fig.update_layout(
    title_text = 'Price by transmission type',
    template = 'plotly_dark'
)

In [ ]:
fig = go.Figure()

to_plot = data_price_float.groupby(by='fuelType', as_index=False).mean()

fig.add_trace(go.Bar(
    y = to_plot['fuelType'],
    x = to_plot['price'],
    orientation='h'
))

fig.update_layout(
    title_text = 'Price by fuel type',
    template = 'plotly_dark'
)

In [ ]:
fig = go.Figure()

to_plot = data_price_float.groupby(by='brand', as_index=False).mean()

fig.add_trace(go.Bar(
    y = to_plot['brand'],
    x = to_plot['price'],
    orientation='h'
))

fig.update_layout(
    title_text = 'Price by brand',
    template = 'plotly_dark'
)

# Machine Learning

In [ ]:
X_all = data.drop(['price'], axis=1)
y_all = data['price']

## Dummy variables

In [ ]:
# Create dummy variables
columns = ['model', 'transmission', 'fuelType', 'brand']

dummy = pd.get_dummies(X_all[columns], dtype=np.int32)
X_dummy = pd.concat([X_all, dummy], axis=1)
X_dummy.drop(columns, axis=1, inplace=True)

In [ ]:
X_dummy

## Split data into train and test set

In [ ]:
X, X_test, y, y_test = train_test_split(X_dummy, y_all, test_size=0.1, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=43)

In [ ]:
X_train.shape

In [ ]:
X_val.shape

In [ ]:
X_test.shape

# Model

## Linear Regression

In [ ]:
reg_model = LinearRegression()
reg_model.fit(X_train, y_train)

In [ ]:
y_hat = reg_model.predict(X_val)
r2_score(y_val, y_hat)

## Random Forest Regressor

In [ ]:
forest_model = RandomForestRegressor()
forest_model.fit(X_train, y_train)

In [ ]:
y_hat_forest = forest_model.predict(X_val)
r2_score(y_val, y_hat_forest)

# Final prediction 

In [ ]:
y_pred = forest_model.predict(X_test)
r2_score(y_test, y_pred)